# Demonstration of AWS CLI:

- [configure](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html)
- [documentation](https://docs.aws.amazon.com/cli/latest/reference/)

#### check instalation:

In [ ]:
! aws --version
! type aws

#### show help options: 

In [ ]:
! aws s3 sync help

#### who we are?

In [ ]:
! aws sts get-caller-identity

#### debugging...

In [ ]:
! aws sts get-caller-identity --debug

In [ ]:
! aws sts get-caller-identity --debug 2>&1 | grep "credentials"

#### listing regions:

- json, text, yaml, table [formats](https://docs.aws.amazon.com/cli/latest/userguide/cli-usage-output-format.html)

In [ ]:
! aws ec2 describe-regions --output json

#### test if you have permissions to invoke

In [ ]:
! aws ec2 describe-regions --dry-run

#### filtering, constrains:
 - [jmespath](https://jmespath.org/)
 - [query](https://docs.aws.amazon.com/cli/latest/userguide/cli-usage-filter.html#cli-usage-filter-client-side-advanced)
 

In [ ]:
! aws ec2 describe-regions --query 'Regions[*]'

In [ ]:
! aws ec2 describe-regions --query 'Regions[0]'

In [ ]:
! aws ec2 describe-regions --query "Regions[?starts_with(RegionName, 'eu')]"

#### projections:

In [ ]:
! aws ec2 describe-regions --query 'Regions[*].[RegionName, Endpoint]'

#### use skeleton

- [docs](https://docs.aws.amazon.com/cli/latest/userguide/cli-usage-skeleton.html)

##### Example command:

    ws s3api create-bucket \
    --bucket artsobcz-cli-demo-bucket \
    --create-bucket-configuration LocationConstraint=eu-west-1

In [ ]:
! aws s3api create-bucket --generate-cli-skeleton

In [ ]:
%%bash
tee -a ./create_bucket_args.json << END
{
    "Bucket": "artsobcz-cli-demo-bucket",
    "CreateBucketConfiguration": {
        "LocationConstraint": "eu-west-1"
    }
}
END

In [ ]:
! aws s3api create-bucket --cli-input-json file://create_bucket_args.json --debug

In [ ]:
! aws s3api delete-bucket --bucket artsobcz-cli-demo-bucket

#### assume role:

In [ ]:
! aws iam get-role --role-name s3_demo_role

In [ ]:
! aws iam list-attached-role-policies --role-name s3_demo_role

In [ ]:
%%bash
echo -e "\nwho we are without role:\n"
aws sts get-caller-identity

result=$(aws sts assume-role --role-arn arn:aws:iam::695414717878:role/s3_demo_role --role-session-name test | jq '.')

export AWS_ACCESS_KEY_ID=`echo $result | jq -r ".Credentials.AccessKeyId"`
export AWS_SECRET_ACCESS_KEY=`echo $result | jq -r ".Credentials.SecretAccessKey"`
export AWS_SESSION_TOKEN=`echo $result | jq -r ".Credentials.SessionToken"`

echo -e "\nAWS env variables:\n"
printenv | grep AWS

echo -e "\nwho we are with role:\n"
aws sts get-caller-identity

unset AWS_ACCESS_KEY_ID AWS_SECRET_ACCESS_KEY AWS_ROLE_SESSION_NAME